# Amazon SageMaker Model Monitor と Debugger を使って不正な予測を検知して分析する

### 依存ライブラリのインストール

下のセルを実行して依存ライブラリをインストールしてカーネルを再起動してください

In [ ]:
! pip install imageio opencv-python 'sagemaker>=2,<3' smdebug torchvision
! git clone https://github.com/advboxes/AdvBox advbox
! cd advbox; python setup.py build; python setup.py install
! pip install future

モジュールをImportできるかテストします。

エラーが発生した場合はカーネルを再起動してみてください。

In [ ]:
%load_ext autoreload
%autoreload 2
import advbox

### モデルをアーカイブしてAmazon S3にアップロードします

このノートブックは、43のクラスで構成される [German Traffic Sign dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset)で学習されたResNet18モデルを使用します。 モデルをSageMakerにデプロイする前に、その重みをアーカイブしてAmazonS3にアップロードする必要があります。 

In [ ]:
!mkdir model
!wget https://github.com/aws-samples/amazon-sagemaker-analyze-model-predictions/raw/master/model/model.pt -O model/model.pt

In [ ]:
import tarfile

with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('model', recursive=True)

In [ ]:
import sagemaker
import boto3

boto_session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=boto_session)

inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')
print(inputs)

### PyTorchモデルとデプロイスクリプトを定義

[SageMaker hosting services](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/how-it-works-deployment.html)を使用して、モデルから予測を取得するための永続的なエンドポイントを設定します。 このためにmodel_data引数にモデルをアーカイブしたS3 PathをとるPyTorchModelオブジェクトを定義します。
entry_pointには、model_fn関数とtransform_fn関数が含まれるpretrained_model.pyを定義します。これらの関数はホスティング中に使用され、モデルが推論コンテナ内で正しく読み込まれ、リクエストを適切に処理できるようにします。

In [ ]:
from sagemaker.pytorch import PyTorchModel

role = sagemaker.get_execution_role()

sagemaker_model = PyTorchModel(
    model_data=f's3://{sagemaker_session.default_bucket()}/model/model.tar.gz',
    role=role,
    source_dir='code',
    entry_point='pretrained_model.py',
    framework_version='1.3.1',
    py_version='py3',
)

`pretrained_model.py`を見てみましょう

In [ ]:
%pycat code/pretrained_model.py

### SageMaker Model Monitorのセットアップとモデルのデプロイ

[SageMaker Model Monitor](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/model-monitor.html)は、本番環境の機械学習モデルを自動的に監視し、データ品質の問題を検出するとアラートを出します。 エンドポイントの入力と出力をキャプチャし、後でModel Monitorが収集したデータとモデルの予測を検査できるように監視スケジュールを作成します。

DataCaptureConfig APIは、Model Monitorが出力先のAmazon S3バケットに保存する入力と出力の割合を指定します。 この例では、サンプリングの割合が50％に設定されています。

In [ ]:
from sagemaker.model_monitor import DataCaptureConfig

data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=50,
    destination_s3_uri=f's3://{sagemaker_session.default_bucket()}/endpoint/data_capture',
)

これで、エンドポイントを`ml.m5.xlarge`インスタンスにデプロイする準備が整いました。

In [ ]:
predictor = sagemaker_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',
    data_capture_config=data_capture_config,
    # エンドポイントは、デフォルトで期待されるnumpyではなくJSONを返します
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)

endpoint_name = predictor.endpoint_name

### 正常のテストデータで推論を実行する

推論を実行する前に、[German Traffic Sign dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) のテスト画像をダウンロードします。

In [ ]:
! wget -N https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_Images.zip
! unzip -oq GTSRB_Final_Test_Images.zip
! wget -N https://raw.githubusercontent.com/aditbiswas1/P2-traffic-sign-classifier/master/signnames.csv

画像クラスの名前をロードします。

In [ ]:
from pandas.io.parsers import read_csv

signnames = read_csv('signnames.csv').values[:, 1]
signnames

次に、シリアル化された入力画像を含むペイロードを使用してエンドポイントを呼び出します。 エンドポイントは、transform_fn関数を呼び出して、推論を実行する前にデータを前処理します。エンドポイントはjson文字列にエンコードされた整数のlistとして、画像ストリームの予測クラスを返します。

In [ ]:
%%time
from PIL import Image
import glob
import matplotlib.pyplot as plt

ncols = 4  # Plot in multiple columns to save some space

for index, file in enumerate(glob.glob('GTSRB/Final_Test/Images/*ppm')):
    if index >= 20:
        break

    # Load image file to array:
    image = Image.open(file)

    # Invoke the endpoint: (Returns a 2D 1x1 array)
    result = predictor.predict(image)

    # Plot the results:
    ixcol = index % ncols
    if (ixcol == 0):
        plt.show()
        fig = plt.figure(figsize=(ncols*4.5, 4.5))
    plt.subplot(1, ncols, ixcol + 1)
    plt.title(signnames[result[0][0]])
    plt.imshow(image)
    plt.axis('off')

plt.show()

### SageMaker Model Monitorのスケジュールを定義する

次に、SageMaker Model Monitorを使用して、ベースラインを設定し、監視スケジュールを設定する方法について説明します。 
Model Monitorは、制約条件の設定と平均、分位数、標準偏差のような統計量を計算する[baseline](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-create-baseline.html)という組み込みコンテナを提供しています。[monitoring schedule](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-scheduling.html)を起動すると、収集されたデータを検査して、指定された制約条件と比較し、違反している場合にレポートを生成するProcessingジョブを定期的に実行できます。

この例では、単純なモデルのサニティチェックのみを実行するカスタムコンテナを作成します。[evaluationスクリプト](./docker/evaluation.py)は、予測された画像クラスを単純にカウントします。モデルが特定の道路標識を他のクラスよりも頻繁に予測するような場合、または信頼スコアが一貫して低い場合に問題が起きていると捉えます。ここでは特定の画像クラスが50％以上の確率で予測される場合にアラートをあげるようにします。

まず、カスタムコンテナを作成する必要があります。 [dockerfile](./docker/Dockerfile)は、 evaluationスクリプトをエントリポイントファイルとして受け取ります。次のコードセルは、Dockerコンテナをビルドし、Amazon ECRにアップロードします。

**このノートブックをSageMaker Studio内で実行する場合、`docker build`は機能しないため、`docker`コマンドを提供するインスタンスでコマンドを実行する必要があります。**

In [ ]:
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'sagemaker-processing-container'
tag = ':latest'

region = boto3.session.Session().region_name

uri_suffix = 'amazonaws.com'
if region in ['cn-north-1', 'cn-northwest-1']:
    uri_suffix = 'amazonaws.com.cn'
processing_repository_uri = f'{account_id}.dkr.ecr.{region}.{uri_suffix}/{ecr_repository + tag}'

# Create ECR repository and push docker image
!docker build -t $ecr_repository docker
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

SageMakerモデルモニターを定義します。DockerイメージのURIとevaluationスクリプトに必要な環境変数を指定します。

In [ ]:
from sagemaker.model_monitor import ModelMonitor

monitor = ModelMonitor(
    role=role,
    image_uri=processing_repository_uri,
    instance_count=1,
    instance_type='ml.m5.large',
    env={ 'THRESHOLD':'0.5' },
)

次に、[Model Monitor Schedule](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-scheduling.html)を定義してエンドポイントにアタッチします。 このカスタムコンテナは1時間ごとに実行されます。

In [ ]:
from sagemaker.model_monitor import CronExpressionGenerator, MonitoringOutput
from sagemaker.processing import ProcessingInput, ProcessingOutput

destination = f's3://{sagemaker_session.default_bucket()}/endpoint/monitoring_schedule'
processing_output = ProcessingOutput(output_name='model_outputs',
                                     source='/opt/ml/processing/outputs',
                                     destination=destination,
                                    )
output = MonitoringOutput(source=processing_output.source, 
                          destination=processing_output.destination)

monitor.create_monitoring_schedule(
    output=output,
    endpoint_input=predictor.endpoint_name,
    schedule_cron_expression=CronExpressionGenerator.hourly(), # 1hごとに実行
)

監視スケジュールを見てみましょう

In [ ]:
monitor.describe_schedule()

SageMaker Model Monitorは、1時間ごとにProcessingジョブを実行します。 これらのProcessingジョブの実行を一覧表示できます。

In [ ]:
jobs = monitor.list_executions()
jobs

Processingジョブの詳細にアクセスできます。

In [ ]:
if len(jobs) > 0:
    print(monitor.list_executions()[-1].describe())
else:
    print("""No processing job has been executed yet. 
    This means that one hour has not passed yet. 
    You can go to the next code cell and run the processing job manually""")

### Processingジョブの実行

1時間待つ代わりに、手動でProcessingジョブを開始して、いくつかの分析結果を取得できます。 これを行うために、カスタム画像の画像URIを取得するProcessorオブジェクトを定義します。 ジョブの入力は、キャプチャされた推論リクエストとレスポンスが保存されるS3のPathになり、スケジュールされたジョブが書き込むのと同じ宛先に結果を出力します。

In [ ]:
from sagemaker.processing import Processor

processor = Processor(
    role=role,
    image_uri=processing_repository_uri,
    instance_count=1,
    instance_type='ml.m5.large',
    env={ 'THRESHOLD':'0.5' },
)
    
processor.run(
    [ProcessingInput(input_name='data',
                     source=f's3://{sagemaker_session.default_bucket()}/endpoint/data_capture',
                     destination='/opt/ml/processing/input/endpoint/',
                    )],
    [ProcessingOutput(output_name='model_outputs',
                      source='/opt/ml/processing/outputs',
                      destination=destination,
    )],
)

### 予期しないモデルの動作をキャプチャする

In [ ]:
! mkdir adversarial_examples

スケジュールが定義されたので、モデルをリアルタイムで監視する準備が整いました。[AdvBox Toolkit](https://github.com/advboxes/AdvBox)を使用して、データセットから「敵対的」に変更された画像を生成します。この場合、ピクセルレベルの摂動により、モデルがだまされて誤ったクラスが予測されます。 画像は元の画像と視覚的に似ています。 この敵対的なデータをモデルで実行し、監視設定で予期しない動作を識別できることを確認します。

In [ ]:
import utils
from advbox.adversarialbox.adversary import Adversary
from advbox.adversarialbox.attacks.deepfool import DeepFoolAttack
from advbox.adversarialbox.models.pytorch import PytorchModel

model = utils.load_model()

m = PytorchModel(model, None, (-3, 3), channel_axis=1)

attack = DeepFoolAttack(m)
attack_config = { 'iterations': 100, 'overshoot': 0.02 }

dataloader = utils.get_dataloader() # GTSRB/Final_Testからdatasetとdataloaderを作成

敵対的な画像を生成します

次のセルはml.t2.mediumの場合、完了まで15minほどかかります

In [ ]:
%%time
import numpy as np


for index, (inputs, labels) in enumerate(dataloader):

    adversary = Adversary(inputs.cpu().numpy(), None)

    tlabel = 14 #class label for stop sign
    adversary.set_target(is_targeted_attack=True, target_label=tlabel)

    adversary = attack(adversary, **attack_config)

    if adversary.is_successful():

        adv=adversary.adversarial_example[0]

        utils.show_images_diff(inputs, adv, adversary.adversarial_label, signnames, index) # オリジナル、生成画像、差分の表示と生成画像の保存

        if index == 100:
            break

それでは敵対的な画像をエンドポイントに送信してみましょう

In [ ]:
%%time

ncols = 4

for index, file in enumerate(glob.glob('adversarial_examples/*png')):
    # Load image file to array:
    image = Image.open(file)

    # Invoke the endpoint: (Returns a 2D 1x1 array)
    result = predictor.predict(image)

    # Plot the results:
    ixcol = index % ncols
    if (ixcol == 0):
        plt.show()
        fig = plt.figure(figsize=(ncols*4.5, 4.5))
    plt.subplot(1, ncols, ixcol + 1)
    plt.title(signnames[result[0][0]])
    plt.imshow(image)
    plt.axis('off')

plt.show()

### Processing jobを実行する

SageMaker Model Monitorは1時間ごとにProcessingジョブを実行しますが、以前と同様に手動でProcessingジョブを実行することもできます。

これは敵対的な画像の送信をModel Monitorの分析に反映させるためのものです

In [ ]:
processor.run(
    [ProcessingInput(input_name='data',
                     source=f's3://{sagemaker_session.default_bucket()}/endpoint/data_capture',
                     destination='/opt/ml/processing/input/endpoint/',
                    )],
    [ProcessingOutput(output_name='model_outputs',
                      source='/opt/ml/processing/outputs',
                      destination=destination,
                     )],
)


SageMaker Model Monitorは、次のProcessingジョブをスケジュールするときに、Amazon S3でキャプチャおよび保存された予測結果を分析します。 前述のように、Processingジョブは単純なサニティチェックを実行するだけです。予測された画像クラスをカウントするだけで、1つのクラスが50％以上の確率で予測されると、アラートを発生します。advserialイメージをエンドポイントに送信したため、イメージクラス14（「一時停止の標識」）の異常なカウントが表示されます。 SageMaker Studioでジョブのステータスを追跡すると、問題が見つかったことがわかります。

![](images/screenshot.png)

CloudWatchログからさらに詳細を取得できます（/aws/sagemaker/ProcessingJobs内にあります）。Processingジョブは、キーが43の画像クラスであり、値がカウントであるディクショナリを出力します。 たとえば、下の出力では、エンドポイントが画像クラス:9を2つと、画像クラス:14を異常な頻度で予測したことがわかります。このクラスは322回予測されており、しきい値の50％よりも高くなっています。
```
Warning: Class 14 ('Stop sign') predicted more than 80 % of the time which is above the threshold
Predicted classes {9: 2, 19: 1, 25: 1, 14: 322, 13: 5, 5: 1, 8: 10, 18: 1, 31: 4, 26: 8, 33: 4, 36: 4, 29: 20, 12: 8, 22: 4, 6: 4}

```

ディクショナリの値もCloudWatchメトリクスとして保存されるため、CloudWatchコンソールを使用してメトリクスデータのグラフを作成できます。

### エンドポイントを更新してSageMaker Debugger hookを有効にする

Processingジョブが問題を検出したら、モデルについてさらに洞察を得る時が来ました。 エンドポイントの推論関数を変更して、モデルからテンソルを出力します（entry_pointの引数のスクリプトが変更されています）。

In [ ]:
sagemaker_model = PyTorchModel(
    model_data=f's3://{sagemaker_session.default_bucket()}/model/model.tar.gz',
    role=role,
    source_dir='code',
    entry_point='pretrained_model_with_debugger_hook.py',
    framework_version='1.3.1',
    py_version='py3',
)

変更した推論関数を見てみましょう。

In [ ]:
%pycat code/pretrained_model_with_debugger_hook.py

model_fnに[SageMaker Debugger hook configuration](https://github.com/awslabs/sagemaker-debugger/blob/master/docs/api.md#hook)を作成します。これはinclude_regexに出力したいテンソルの名前を示す正規表現を取ります。テンソルは、SageMakerのデフォルトバケットの「endpoint/tensors」に保存されます。

次に、既存のエンドポイントを更新します。これにより、古いentry_pointファイルが新しいファイルに置き換えられます。

In [ ]:
# Model.deploy()のupdate_endpointパラメーターがSageMaker SDKv2で削除されたため、
# ここではいくつかの内部/プライベート関数を使用して、APIバックエンドにモデルを強制的に登録します...

sagemaker_model._init_sagemaker_session_if_does_not_exist('ml.m5.xlarge')
sagemaker_model._create_sagemaker_model('ml.m5.xlarge')

# 次に、エンドポイントを新しく登録されたモデルバージョンにポイントします。
predictor.update_endpoint(
    model_name=sagemaker_model.name,
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',
)

これで推論リクエストが行われるたびに、テンソルが記録され、Amazon S3にアップロードされます。 それでは、さらに敵対的な画像を送信しましょう。

In [ ]:
%%time

ncols = 4

for index, file in enumerate(glob.glob('adversarial_examples/*png')):
    # Load image file to array:
    image = Image.open(file)

    # Invoke the endpoint: (Returns a 2D 1x1 array)
    result = predictor.predict(image)

    # Plot the results:
    ixcol = index % ncols
    if (ixcol == 0):
        plt.show()
        fig = plt.figure(figsize=(ncols*4.5, 4.5))
    plt.subplot(1, ncols, ixcol + 1)
    plt.title(signnames[result[0][0]])
    plt.imshow(image)
    plt.axis('off')

plt.show()

これで顕著性（saliency）を計算して、モデルから視覚的な説明を取得できます。

### SageMaker Debuggerを使用して予測結果を分析する

関連するテンソルをキャプチャするようにSageMaker Debugger hookを構成しました。 エンドポイントが更新され、テンソルがアップロードされたので、[trial](https://github.com/awslabs/sagemaker-debugger/blob/master/docs/analysis.md#creating-a-trial-object)を作成し、Amazon S3からデータを読み取ります。

In [ ]:
from smdebug.trials import create_trial

trial = create_trial(f's3://{sagemaker_session.default_bucket()}/endpoint/tensors')

これらのテンソルから、入力画像のどの領域が予測結果にとって最も重要であるかを説明する顕著性(saliency)マップを計算できます。 [Full-Gradient Representation for Neural Network Visualization [1]](https://arxiv.org/abs/1905.00780)で説明されている方法では、すべての中間特徴とそのバイアスが必要です。 次のセルは、バッチノルム層とダウンサンプリング層の出力の勾配と対応するバイアスを取得します。 ResNet以外のモデルを使用する場合は、次のセルの正規表現を調整する必要がある場合があります。

In [ ]:
biases, gradients = [], []

for tname in trial.tensor_names(regex='.*gradient.*bn.*output|.*gradient.*downsample.1.*output'):
    gradients.append(tname)
    
for tname in trial.tensor_names(regex='^(?=.*bias)(?:(?!fc).)*$'):
    biases.append(tname)

BatchNormレイヤの場合、暗黙的なバイアス(implicit biases)を計算する必要があります。 次のコードセルで、必要なテンソルを取得します。

In [ ]:
bn_weights, running_vars, running_means = [], [], []

for tname in trial.tensor_names(regex='.*running_mean'):
    running_means.append(tname)
    
for tname in trial.tensor_names(regex='.*running_var'):
    running_vars.append(tname)

for tname in trial.tensor_names(regex='.*bn.*weight|.*downsample.1.*weight'):
    bn_weights.append(tname) 

結果を正規化するヘルパー関数を定義します。

In [ ]:
import numpy as np

def normalize(tensor):
    tensor = tensor - np.min(tensor)
    tensor = tensor / np.max(tensor)
    return tensor

BatchNormレイヤの移動平均は、全体的なバイアスを計算するときに考慮する必要がある暗黙的のバイアス(implicit bias)を導入します。

In [ ]:
def compute_implicit_biases(bn_weights, running_vars, running_means, step):
    implicit_biases = []
    for weight_name, running_var_name, running_mean_name in zip(bn_weights, running_vars, running_means):
        weight = trial.tensor(weight_name).value(step_num=step, mode=modes.PREDICT)
        running_var = trial.tensor(running_var_name).value(step_num=step, mode=modes.PREDICT)
        running_mean = trial.tensor(running_mean_name).value(step_num=step, mode=modes.PREDICT)
        implicit_biases.append(- running_mean / np.sqrt(running_var) * weight)
    return implicit_biases

次のコードセルは、すべてのテンソルをフェッチし、画像ごとの顕著性(saliency)マップを計算します。 赤のピクセルは最も関連性の高いピクセルを示し、青のピクセルは画像クラスを予測するための最も関連性の低いピクセルを示します。

In [ ]:
%%time
from smdebug import modes
from smdebug.core.modes import ModeKeys
import cv2
import scipy.ndimage
import scipy.special
import utils

for step in trial.steps():

    image_batch = trial.tensor("ResNet_input_0").value(step_num=step, mode=modes.PREDICT)

    #compute implicit biases from batchnorm layers
    implicit_biases = compute_implicit_biases(bn_weights, running_vars, running_means, step)

    for item in range(image_batch.shape[0]):

        #input image
        image = image_batch[item,:,:,:]

        #get gradients of input image
        image_gradient = trial.tensor("gradient/image").value(step_num=step, mode=modes.PREDICT)[item,:]  
        image_gradient = np.sum(normalize(np.abs(image_gradient * image)), axis=0)
        saliency_map = image_gradient

        for gradient_name, bias_name, implicit_bias in zip(gradients, biases, implicit_biases):

            #get gradients and bias vectors for corresponding step
            gradient = trial.tensor(gradient_name).value(step_num=step, mode=modes.PREDICT)[item:item+1,:,:,:]
            bias = trial.tensor(bias_name).value(step_num=step, mode=modes.PREDICT) 
            bias = bias + implicit_bias

            #compute full gradient
            bias = bias.reshape((1,bias.shape[0],1,1))
            bias = np.broadcast_to(bias, gradient.shape)
            bias_gradient = normalize(np.abs(bias * gradient))

            #interpolate to original image size
            for channel in range(bias_gradient.shape[1]):
                interpolated = scipy.ndimage.zoom(bias_gradient[0,channel,:,:], 128/bias_gradient.shape[2], order=1)
                saliency_map += interpolated 


        #normalize
        saliency_map = normalize(saliency_map) 

        #predicted class and propability
        predicted_class = trial.tensor("fc_output_0").value(step_num=step, mode=modes.PREDICT)[item,:]
        print("Predicted class:", np.argmax(predicted_class))
        scores = np.exp(np.asarray(predicted_class))
        scores = scores / scores.sum(0)

        #plot image and heatmap
        utils.plot_saliency_map(saliency_map, image, np.argmax(predicted_class), str(int(np.max(scores) * 100)), signnames )

### Cleanup

以下のセルで削除できない場合はコンソールから削除してください。

In [ ]:
import time

monitor.delete_monitoring_schedule()

In [ ]:
time.sleep(60) # actually wait for the deletion

In [ ]:
predictor.delete_endpoint()